## AgentCore Evaluations - Strands エージェントのオンデマンド評価

このチュートリアルでは、Strands エージェントに適用される AgentCore Evaluations のオンデマンド評価の使用方法を学びます。

このラボを実行するには、まず [00-prereqs](../../00-prereqs) フォルダのコードを使用して Strands エージェントを作成し、[01-creating-custom-evaluators](../../01-creating-custom-evaluators) のコードを使用してカスタム評価メトリクスを作成しておく必要があります

### 学習内容
- AgentCore Starter toolkit を使用してトレースにオンデマンド評価を実行する方法

### チュートリアル詳細

| 情報               | 詳細                                                                          |
|:-------------------|:------------------------------------------------------------------------------|
| チュートリアルタイプ | オンデマンド評価メトリクス（ビルトインおよびカスタム）を使用した Strands エージェントの評価 |
| チュートリアル構成   | ビルトインおよびカスタム評価メトリクスでの評価実行                               |
| チュートリアル分野   | 横断的                                                                        |
| 複雑さ              | 簡単                                                                          |
| 使用 SDK            | Amazon Bedrock AgentCore Starter toolkit                                      |

### オンデマンド評価

オンデマンド評価は、選択したスパンのセットを直接分析することで、特定のエージェントインタラクションを柔軟に評価する方法を提供します。本番トラフィックを継続的に監視するオンライン評価とは異なり、オンデマンド評価では選択したインタラクションに対してターゲットを絞った評価をいつでも実行できます。

オンデマンド評価では、スパン、トレース、またはセッション ID を提供することで、評価したい正確なスパン、トレース、またはセッションを指定します。AgentCore Starter toolkit を使用すると、セッション内のすべてのトレースを自動的に評価することもできます。

カスタム評価メトリクスまたはビルトイン評価メトリクスをエージェントのインタラクションに適用できます。この評価タイプは、特定の顧客インタラクションを調査する必要がある場合、報告された問題の修正を検証する場合、または品質改善のために履歴データを分析する場合に特に便利です。評価リクエストを送信すると、サービスは指定されたスパンのみを処理し、分析用の詳細な結果を提供します。

### AgentCore Observability でエージェントからトレースを生成

AgentCore Observability は、[OpenTelemetry (OTEL)](https://opentelemetry.io/) トレースを活用して詳細な実行データをキャプチャし構造化することで、呼び出し中のエージェントの動作を包括的に可視化します。AgentCore は [AWS Distro for OpenTelemetry (ADOT)](https://aws-otel.github.io/) を利用して、さまざまなエージェントフレームワーク間で異なるタイプの OTEL トレースを計装します。

エージェントが AgentCore Runtime でホストされている場合（このチュートリアルのエージェントのように）、AgentCore Observability の計装は最小限の設定で自動的に行われます。必要なのは `requirements.txt` に `aws-opentelemetry-distro` を含めることだけで、AgentCore Runtime が OTEL 設定を自動的に処理します。エージェントが AgentCore Runtime で実行されていない場合は、AgentCore Observability で利用できるようにするために ADOT で計装する必要があります。テレメトリデータを CloudWatch に送信するための環境変数を設定し、OpenTelemetry 計装でエージェントを実行する必要があります。

プロセスは以下のようになります:

![session_traces](../../images/observability_traces.png)

セッショントレースが AgentCore Observability で利用可能になると、AgentCore Evaluations を使用してエージェントの動作を評価できます。

### オンデマンド評価がトレースとどのように連携するか

オンデマンド評価では、エージェントが呼び出されて AgentCore Observability にトレースが生成されます。これらのトレースはセッションにマッピングされ、ログは Amazon CloudWatch Log グループで利用可能になります。オンデマンド評価では、開発者がどのセッションまたはトレースを使用するかを決定し、トレースの内容を評価するメトリクスとともに AgentCore Evaluations に入力として送信します。プロセスは以下のようになります:


![session_traces](../../images/on_demand_evaluations.png)

### 前のチュートリアルからの情報の取得

このチュートリアルでは、前提条件チュートリアルで AgentCore Runtime にデプロイした Strands エージェントを使用します。事前構築されたメトリクスと、`01-creating-custom-metrics` チュートリアルで作成した `response_quality` メトリクスで評価します。エージェントと評価メトリクスの情報を取得しましょう。

In [ ]:
%store -r launch_result_strands
%store -r session_id_strands
%store -r evaluator_id
try:
    print("Agent Id:", launch_result_strands.agent_id)
    print("Agent ARN:", launch_result_strands.agent_arn)
except NameError as e:
    raise Exception("""Missing launch results from your Strands agent. Please run 00-prereqs before executing this lab""")

try:
    print("Session id:", session_id_strands)
except NameError as e:
    raise Exception("""Missing session id from your Strands agent. Please run 00-prereqs before executing this lab""")

try:
    print("Evaluator id:", evaluator_id)
except NameError as e:
    raise Exception("""Missing custom evaluator id. Please run 01-creating-custom-evaluators before executing this lab""")

### AgentCore Evaluations クライアントの初期化

それでは AgentCore Starter toolkit から AgentCore Evaluations クライアントを初期化しましょう。

In [ ]:
from bedrock_agentcore_starter_toolkit import Evaluation, Observability
import os
import json
from boto3.session import Session
from IPython.display import Markdown, display

In [ ]:
boto_session = Session()
region = boto_session.region_name
print(region)

In [ ]:
eval_client = Evaluation(region=region)

### 評価の実行

AgentCore Evaluations を実行するには、セッション、トレース、またはスパン情報を提供する必要があります。前のチュートリアルで見たように、異なるメトリクスはエージェントトレースから異なるレベルの情報を必要とします。

![metrics level](../../images/metrics_per_level.png)

AWS の SDK のいずれかを使用している場合、トレースを自分で処理する必要があります。AgentCore Starter toolkit はこのプロセスを簡素化し、セッション ID またはトレース ID に基づいてトレースを処理します。これにより、`run` インタラクション中に AgentCore starter toolkit 評価クライアントにセッション ID を提供すると、SDK がそのセッション内のすべてのトレースを抽出し、トレースおよびスパンレベルのメトリクスで評価します。オプションで、評価結果を保存するための出力ファイル名も提供できます。

### Goal Success Rate

エージェントの Goal Success Rate を評価しましょう。エージェントには以下の質問をしたことを覚えておいてください:

* What is the weather now?
* How much is 2+2?
* Can you tell me the capital of the US?

In [ ]:
goal_sucess_results = eval_client.run(
    agent_id=launch_result_strands.agent_id,
    session_id=session_id_strands, 
    evaluators=["Builtin.GoalSuccessRate"]
)

評価メトリクスの結果を理解しましょう。結果オブジェクトには、実行された評価についての情報（session_id、trace_id、input_data）と評価結果が含まれています。

評価結果には、評価メトリクス情報（id、name、ARN）、評価値、評価ラベル、評価説明、評価ジョブに関する追加のコンテキスト（spanContext、token_usage など）が含まれます。

評価レスポンスを見てみましょう

In [ ]:
for result in goal_sucess_results.results:
    information = f"""
    Goal Success: {result.label} ({result.value})
    Explanation: \n{result.explanation}]\n
    Token Usage: {result.token_usage}\n
    Context: {result.context}\n
    """
    display(Markdown(information))

### Correctness

同じセッションをトレースレベルのメトリクス Correctness で分析しましょう

In [ ]:
correctness_results = eval_client.run(
    agent_id=launch_result_strands.agent_id,
    session_id=session_id_strands, 
    evaluators=["Builtin.Correctness"]
)

評価メトリクスの結果を理解しましょう。この場合、Correctness はトレースレベルで評価されるため、各トレースはそれぞれ独自の評価結果を取得します。

In [ ]:
for result in correctness_results.results:
    information = f"""
    Correctness: {result.label} ({result.value})
    Explanation: \n{result.explanation}]\n
    Token Usage: {result.token_usage}\n
    Context: {result.context}\n
    """
    display(Markdown(information))
    print("================================================")

### Tool selection accuracy と parameter selection accuracy

ツールとパラメータ選択についてエージェントを評価しましょう。両方のメトリクスはスパンレベルで評価されます。

In [ ]:
parameter_results = eval_client.run(
    agent_id=launch_result_strands.agent_id,
    session_id=session_id_strands, 
    evaluators=["Builtin.ToolParameterAccuracy", "Builtin.ToolSelectionAccuracy"]
)

結果を分析しましょう。この場合、同じ run で2つの異なるメトリクスでセッションを評価しています。つまり、どの評価メトリクスが各レスポンスを生成しているかを知る必要があります。これは結果の `evaluator_name` プロパティで確認できます。エージェントがツールをどれだけうまく使用したか見てみましょう:

In [ ]:
for result in parameter_results.results:
    information = f"""
    Metric: {result.evaluator_name}
    Value: {result.label} ({result.value})
    Explanation: \n{result.explanation}]\n
    Token Usage: {result.token_usage}\n
    Context: {result.context}\n
    """
    display(Markdown(information))
    print("================================================")

### カスタム評価メトリクスの使用

セッション、トレース、スパンレベルで評価メトリクスを使用したので、カスタムメトリクスを使用して応答品質を評価しましょう:

In [ ]:
custom_results = eval_client.run(
    agent_id=launch_result_strands.agent_id,
    session_id=session_id_strands, 
    evaluators=[evaluator_id]
)

評価結果を見てみましょう。この場合、以下の指示を持つエージェントを評価しています:

```
You're a helpful assistant. You can do simple math calculation, and tell the weather.
```

評価メトリクスでは、評価指示に記載されているように、スコープ外の応答に対して `Very Poor` の品質でエージェントにペナルティを与えています:

```
...
**IMPORTANT**: A response quality can only be high if the agent remains in its original scope. Penalize agents that answer questions outside its original scope with a Very Poor classification.
...
```

以下の質問を評価しているため:

* What is the weather now?
* How much is 2+2?
* Can you tell me the capital of the US?

最後の質問に対してエージェントが `Very Poor` の評価を受けることを期待しています。

In [ ]:
for result in custom_results.results:
    information = f"""
    Metric: {result.evaluator_name}
    Value: {result.label} ({result.value})
    Explanation: \n{result.explanation}]\n
    Token Usage: {result.token_usage}\n
    Context: {result.context}\n
    """
    display(Markdown(information))
    print("================================================")

### 評価結果の保存

AgentCore starter toolkit は、構造化された出力ファイルにエージェント評価の結果を保存するのにも役立ちます。run 中に `ouput` パラメータを提供するだけです。

In [ ]:
save_results = eval_client.run(
    agent_id=launch_result_strands.agent_id,
    session_id=session_id_strands, 
    evaluators=[
        evaluator_id
    ],
    output="evals_results/output.json"
)

### おめでとうございます！

オンデマンド機能でエージェントを評価しました。次のチュートリアルでは、オンライン評価メトリクスを設定してエージェントに接続することで、本番環境でのエージェント評価を自動化します。